## 1️⃣ Load data

In [24]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_time_df = train_df['TIMESTAMP']
test_time_df = test_df['TIMESTAMP']



#train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'TIMESTAMP'])
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

done
done


In [23]:
def split_PC(traindf, test_df):

    pc_v = sorted(train_df['PRODUCT_CODE'].value_counts().index)

    # train
    for idx, i in enumerate(pc_v):
        globals()['pc{}'.format(idx+1)]=train_x[train_x['PRODUCT_CODE']==i]

    # train y
    for idx, i in enumerate(pc_v):
        globals()['pc{}_y'.format(idx+1)]=train_x['Y_Class'][train_x['PRODUCT_CODE']==i]

    # test
    for idx, i in enumerate(pc_v):
        globals()['pc{}_test'.format(idx+1)]=test_x[test_x['PRODUCT_CODE']==i]

    print('done')

def split_line(train_df, test_df):

    
    line_v = sorted(train_df['LINE'].value_counts().index)

    # train
    for idx, i in enumerate(line_v):
        globals()['l{}'.format(idx+1)]=train_x[train_x['LINE']==i]

    # train y
    for idx, i in enumerate(line_v):
        globals()['l{}_y'.format(idx+1)]=train_x['Y_Class'][train_x['LINE']==i]

    # test
    for idx, i in enumerate(line_v):
        globals()['l{}_test'.format(idx+1)]=test_x[test_x['LINE']==i]

    print('done')

split_PC(train_df, test_df)
split_line(train_df, test_df)

In [17]:
all_line_list = [l1, l2, l3, l4, l5, l6, l1_test, l2_test, l3_test, l4_test, l5_test, l6_test]
train_line_list = [l1, l2, l3, l4, l5, l6]


def line_fillna(all, train):

    # train drop : Y_Class
    for l in train:
        l.drop(['Y_Class'], axis=1, inplace=True)
    
    
    # fillna : 0
    for l in all:
        l.fillna(0, inplace=True)

    # MinMaxScaler
    from sklearn.preprocessing import StandardScaler
    
    for l in all:
        sts = StandardScaler()
        sts.fit(l.drop(['LINE','PRODUCT_CODE'], axis=1))
        l = sts.transform(l.drop(['LINE','PRODUCT_CODE'], axis=1))
      #  l =pd.concat([l[['LINE','PRODUCT_CODE']], mmx.transform(l.drop(['LINE','PRODUCT_CODE'], axis=1))])

    # labelEncorder
    from sklearn.preprocessing import LabelEncoder

        # 1) Line
    for l in all:
        le = LabelEncoder()
        le.fit(['T100304', 'T100306', 'T050304', 'T010306', 'T010305', 'T050307'])
        l['LINE'] = le.transform(l['LINE'])
        
        # 2) PRODUCT_CODE
    for l in all:
        le = LabelEncoder()
        le.fit(['T_31', 'A_31', 'O_31'])
        l['PRODUCT_CODE'] = le.transform(l['PRODUCT_CODE'])        

    print('done')

line_fillna(all_line_list, train_line_list)

done


In [ ]:
train_x_pc_list = [pc1, pc2, pc3]
train_y_pc_list = [pc1_y, pc2_y, pc3_y]
test_pc_list = [pc1_test, pc2_test, pc3_test]


def PC_model_fit_predeict(train_x_pc_list):

    from sklearn.ensemble import RandomForestClassifier

    # 1) define model, 2) fit, 3) predict
    for idx ,i in enumerate(train_x_pc_list):
        
        # define model
        rfc = RandomForestClassifier(random_state=42)
        
        # fit
        rfc.fit(i, train_y_pc_list[idx])

        # predict
        globals()['pred_pc{}'.format(idx+1)] = rfc.predict(test_pc_list[idx])
        globals()['df_pred_pc{}'.format(idx+1)] = pd.DataFrame(data = globals()['pred_pc{}'.format(idx+1)],
                                                            columns=['Y_Class'],
                                                            index=test_pc_list[idx].index)
        

    df_pred = [df_pred_pc1, df_pred_pc2, df_pred_pc3]

    submit_array = pd.concat(df_pred, axis=0).sort_index()

    return submit_array


#def submit_line():
    
submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = PC_model_fit_predeict(train_x_pc_list)['Y_Class']
submit_csv.to_csv('split_PC_submission.csv', index=False)